## Import of required Modules
Make sure, that the imported moduls are installed.

In [1]:
import pickle
import re
from tqdm import tqdm
import time
import numpy as np
from IPython.display import clear_output 
import pandas as pd
import time
import os

current_path = os.getcwd()
current_path

In [2]:
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service

wait_time = 0.25

Retrieve Links

In [16]:
chrome_path = r'driver/chromedriver.exe' 
chrome_options = Options()

chrome_options.binary_location = f'{current_path}\\chrome-win64\\chrome.exe'
chrome_options.add_argument("--headless") # open chrome in background

links = []

driver = webdriver.Chrome(service=Service(r'driver/chromedriver.exe') , options=chrome_options)
driver.get('https://www.allsides.com/headline-roundups')
wait = WebDriverWait(driver, 10)

# retrieve number of pages 
last_page_button = driver.find_element(By.CLASS_NAME, "pager-last")
link_last_page = last_page_button.find_elements(By.TAG_NAME, "a")
t = link_last_page[0].get_attribute("href")
last_page_index = int(t[-3:])

WebDriverWait(driver, 20)

for page in tqdm(range(2,last_page_index+1)):   

    driver.get("https://www.allsides.com/headline-roundups?page="+str(page))
    WebDriverWait(driver, 20)

    main_table = driver.find_element(By.XPATH, "//*[@id=\"block-views-de37fa32ea86f5545eb9b7722977a70d\"]/div/div[2]/table/tbody") # table body
    rows = main_table.find_elements(By.TAG_NAME, "tr")

    for i in rows:
        entry = i.find_elements(By.TAG_NAME, "td")
        link = entry[0].find_element(By.TAG_NAME, "a")
        links.append(link.get_attribute("href"))

    clear_output()
    
driver.close()

  1%|          | 1/191 [00:14<46:34, 14.71s/it]


KeyboardInterrupt: 

In [60]:
# exports links as pickle file
with open("linklist_allsides_news.pickle", "wb") as f:
    pickle.dump(links, f, pickle.HIGHEST_PROTOCOL)

In [61]:
# exports links as csv
with open("linklist_allsides_news.csv", "w") as f:
    for line in links:
        print(line, file=f)

In [7]:
def check_exists_by_xpath(xpath):
    try:
        driver.find_elements(By.XPATH, xpath)[0]
    except:
        return False
    return True


Retrieve Articles

In [8]:
# load link list from pickle file
with open("linklist_allsides_news.pickle", "rb") as f:
    links = pickle.load(f)

In [17]:
chrome_path = r'driver/chromedriver.exe' 
chrome_options = Options()

chrome_options.binary_location = f'{current_path}\\chrome-win64\\chrome.exe'
chrome_options.add_argument("--headless")

# list for results 
data = []

# retrieve information from articles in list of links
count = 0
for li in tqdm(links):

    ### For Backup incase of error ###
    # if count <= 7801:
    #     count +=1 
    #     continue
    
    # if count % 100 == 1:
    #     df_backup = pd.DataFrame(data)
    #     df_backup.to_csv(f"allsides_news_{count}.csv")

    start = time.time()

    time.sleep(wait_time)
    driver = webdriver.Chrome(service=Service(r'driver/chromedriver.exe') , options=chrome_options)
    print(li)
    
    driver.get(li)
    
    try:
        heading = driver.find_element(By.TAG_NAME, "h1").text
    except:
        print("no heading found")
    
    print(heading)
    
    try:
        date = driver.find_element(By.CLASS_NAME, "date-display-single").text
    except:
        date = ""
    
    try:
        tags = [a.text for a in driver.find_element(By.CLASS_NAME, "page-tags").find_elements(By.TAG_NAME, "a")]
    except:
        tags = ""
        
    divs = ["/html/body/div[3]/div/div/div/div[4]/div/div/div/div[1]", "/html/body/div[3]/div/div/div/div[4]/div/div/div/div[2]", "/html/body/div[3]/div/div/div/div[4]/div/div/div/div[3]"]
   
    for d in divs:
        # to check if div exists 
        try:
            div = driver.find_elements(By.XPATH, d)[0]
        except:
            print("div not found")
            continue
            
        # retrieve link to original article
        try:
            link = div.find_element(By.TAG_NAME, "a").get_attribute("href")
            print(link)
        except:
            print("no link found")

        # This image contains bias information
        try:
            label_img_src = div.find_element(By.XPATH, f"{d}/a[2]/img").get_attribute("src")
        except:
            print('no label image found')

        time.sleep(0.2)
        if "bias-left" in label_img_src:
            print("left")
            try:
                left_heading = div.find_element(By.CLASS_NAME, "news-title").text # heading       
            except:
                left_heading = ""
                print("no headline found")
            try:
                left_source = div.find_element(By.CLASS_NAME, "source-area").find_element(By.TAG_NAME, "span").text #source
            except:
                left_source = ""
                print("no source found")
            try:
                left_text = div.find_element(By.CLASS_NAME, "news-body").find_element(By.CLASS_NAME, "body-contents").text # news text body-contents
            except:
                left_text = ""
                print("no text found")
                
            # add the article information
            data.append({"url":link, "date":date, "title": heading, "tags": tags, "heading":left_heading, "source": left_source, "text": left_text, "bias_rating": "left"})

        if "bias-leaning-left" in label_img_src:
            print("leaning left")
            try:
                left_heading = div.find_element(By.CLASS_NAME, "news-title").text # heading       
            except:
                left_heading = ""
                print("no headline found")
            try:
                left_source = div.find_element(By.CLASS_NAME, "source-area").find_element(By.TAG_NAME, "span").text #source
            except:
                left_source = ""
                print("no source found")
            try:
                left_text = div.find_element(By.CLASS_NAME, "news-body").find_element(By.CLASS_NAME, "body-contents").text # news text body-contents
            except:
                left_text = ""
                print("no text found")
                
            # add the article information
            data.append({"url":link, "date":date, "title": heading, "tags": tags, "heading":left_heading, "source": left_source, "text": left_text, "bias_rating": "leaning left"})


        elif "bias-right" in label_img_src:
            print("right")
            try:
                right_heading = div.find_element(By.CLASS_NAME, "news-title").text # heading
            except:
                right_heading = ""
                print("no headline found")
            try:
                right_source = div.find_element(By.CLASS_NAME, "source-area").find_element(By.TAG_NAME, "span").text #source 
            except:
                right_source = ""
                print("no source found")
            try:
                right_text = div.find_element(By.CLASS_NAME, "news-body").find_element(By.CLASS_NAME, "body-contents").text # news text
            except:
                right_text = ""
                print("no text found")
            
            # add the article information
            data.append({"url":link, "date":date, "title": heading, "tags": tags, "heading":right_heading, "source": right_source, "text": right_text, "bias_rating": "right"}) 

        elif "bias-leaning-right" in label_img_src:
            print("leaning right")
            try:
                right_heading = div.find_element(By.CLASS_NAME, "news-title").text # heading
            except:
                right_heading = ""
                print("no headline found")
            try:
                right_source = div.find_element(By.CLASS_NAME, "source-area").find_element(By.TAG_NAME, "span").text #source 
            except:
                right_source = ""
                print("no source found")
            try:
                right_text = div.find_element(By.CLASS_NAME, "news-body").find_element(By.CLASS_NAME, "body-contents").text # news text
            except:
                right_text = ""
                print("no text found")
            
            # add the article information
            data.append({"url":link, "date":date, "title": heading, "tags": tags, "heading":right_heading, "source": right_source, "text": right_text, "bias_rating": "leaning right"}) 

        else:
            print("center")
            try:
                center_heading = div.find_element(By.CLASS_NAME, "news-title").text # heading
            except:
                center_heading = ""
                print("no headline found")
            try:
                center_source = div.find_element(By.CLASS_NAME, "source-area").find_element(By.TAG_NAME, "span").text #source 
            except:
                center_source = ""
                print("no source found")
            try:
                center_text = div.find_element(By.CLASS_NAME, "news-body").find_element(By.CLASS_NAME, "body-contents").text # news text
            except:
                center_text = ""
                print("no text found")
                
            # add the article information
            data.append({"url":link, "date":date, "title": heading, "tags": tags, "heading":center_heading, "source": center_source, "text": center_text, "bias_rating": "center"})
            
    count += 1 
    
    driver.close()
    clear_output()
   
    time.sleep(wait_time)

  2%|▏         | 1/50 [00:32<26:55, 32.96s/it]


KeyboardInterrupt: 

In [ ]:
df = pd.DataFrame(data)
df.to_csv("allsides_news_complete.csv", index=False)